In [11]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks, peak_prominences

In [12]:
dg_df = pd.read_csv('./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/Demographics.csv')
dg_df.head(16)

,ID,Gender,HbA1c
0,13,MALE,5.7
1,1,FEMALE,5.5
2,3,FEMALE,5.9
3,4,FEMALE,6.4
4,5,FEMALE,5.7
5,2,MALE,5.6
6,6,FEMALE,5.8
7,7,FEMALE,5.3
8,8,FEMALE,5.6
9,10,FEMALE,6.0


In [13]:
###### ACC
# 가속도 벡터 magnitude 계산 함수
def calculate_speed(acc_x, acc_y, acc_z):
    speed_mag = (acc_x ** 2 + acc_y ** 2 + acc_z ** 2) ** 0.5
    return speed_mag

###### EDA
# rolling window에서 피크를 찾는 함수
def find_peaks_in_rolling_window(data, window_size):
    number_of_windows = 0
    peaks_in_window = 0
    for window_data in data.rolling(window=window_size):
        peaks, _ = find_peaks(window_data[' eda'], height=0, distance=4, prominence=0.3)
        if peaks.any():
            peaks_in_window += len(peaks)
        number_of_windows += 1
    return number_of_windows, peaks_in_window

In [14]:
# 빈 DataFrame 생성
df = pd.DataFrame(columns=[
    'calories2hr',
    'protein2hr',
    'sugar2hr',
    'carbs2hr',
    'calories8hr',
    'protein8hr',
    'sugar8hr',
    'carbs8hr',
    'calories24hr',
    'protein24hr',
    'sugar24hr',
    'carbs24hr',
    'acc_mean',
    'acc_std',
    'acc_min',
    'acc_max',
    'acc_q1g',
    'acc_q3g',
    'acc_skew',
    'eda_mean',
    'eda_std',
    'eda_min',
    'eda_max',
    'eda_q1g',
    'eda_q3g',
    'eda_skew',
    'PeakEDA',
    'PeakEDA2hr_sum',
    'PeakEDA2hr_mean',
    'hr_mean',
    'hr_std',
    'hr_min',
    'hr_max',
    'hr_q1g',
    'hr_q3g',
    'hr_skew',
    'maxHRV',
    'minHRV',
    'medianHRV',
    'meanHRV',
    'SDNN',
    'NN50',
    'pNN50',
    'RMSSD',
    'temp_mean',
    'temp_std',
    'temp_min',
    'temp_max',
    'temp_q1g',
    'temp_q3g',
    'temp_skew'
])

## Warnings

* 003 데이터셋 Food_Log에 칼럼 누락되어 있음 추가 필요
* 001 데이터셋의 HR 대해서 datetime 포맷이 다름 (hr_df['datetime']=pd.to_datetime(hr_df['datetime'], format='%m/%d/%y %H:%M'))
* 012 데이터셋 Food_Log의 마지막 행에 데이터 누락됨
2020-05-11,07:00:00,,,Boost,,,,654.0,82.0,5.6,40.0,26.0,26.0
-> 2020-05-11,07:00:00, 2020-05-11 07:00:00,,Boost,,,,654.0,82.0,5.6,40.0,26.0,26.0
* 013 데이터셋 Food_Log의 date와 datetime 포맷이 다름

In [15]:
for id in range(1, len(dg_df)+1):
    fl_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/Food_Log_{id:03}.csv')
    acc_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/ACC_{id:03}.csv')
    eda_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/EDA_{id:03}.csv')
    bvp_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/BVP_{id:03}.csv')
    hr_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/HR_{id:03}.csv')
    ibi_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/IBI_{id:03}.csv')
    temp_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/TEMP_{id:03}.csv')
    dexcom_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/Dexcom_{id:03}.csv')

    biological_sex = dg_df.loc[dg_df['ID'] == id, 'Gender'].values[0]
    hbA1c = dg_df.loc[dg_df['ID'] == id, 'HbA1c'].values[0]

    target = 0

    if hbA1c > 5.6:
        target = 1

    print(f'ID : {id} | target : {target}')
    

    ###### food log
    
    #if id == 13:
    #    fl_df['date']=pd.to_datetime(fl_df['date'], format='%m/%d/%y')
    #else:
    #    fl_df['date']=pd.to_datetime(fl_df['date'])
        
    if id == 13:
        fl_df['time_begin']=pd.to_datetime(hr_df['time_begin'], format='%m/%d/%y %H:%M')
    else:
        fl_df['time_begin']=pd.to_datetime(fl_df['time_begin'])
    fl_df.index = pd.DatetimeIndex(fl_df["time_begin"])
    fl_df = fl_df.sort_index()

    calories2hr = 0.
    protein2hr = 0.
    sugar2hr = 0.
    carbs2hr = 0.
    calories8hr = 0.
    protein8hr = 0.
    sugar8hr = 0.
    carbs8hr = 0.
    calories24hr = 0.
    protein24hr = 0.
    sugar24hr = 0.
    carbs24hr = 0.

    for window_data in fl_df.rolling(window='2H', on='time_begin'):
        calories2hr += window_data['calorie'].values.sum()
        protein2hr += window_data['protein'].values.sum()
        sugar2hr += window_data['sugar'].values.sum()
        carbs2hr += window_data['total_carb'].values.sum()

    for window_data in fl_df.rolling(window='8H', on='time_begin'):
        calories8hr += window_data['calorie'].values.sum()
        protein8hr += window_data['protein'].values.sum()
        sugar8hr += window_data['sugar'].values.sum()
        carbs8hr += window_data['total_carb'].values.sum()

    for window_data in fl_df.rolling(window='24H', on='time_begin'):
        calories24hr += window_data['calorie'].values.sum()
        protein24hr += window_data['protein'].values.sum()
        sugar24hr += window_data['sugar'].values.sum()
        carbs24hr += window_data['total_carb'].values.sum()


    ###### accelerometer

    acc_df['datetime']=pd.to_datetime(acc_df['datetime'])
    acc_df['vector_mag'] = calculate_speed(acc_df[' acc_x'], acc_df[' acc_y'], acc_df[' acc_z'])
    acc_resampled = acc_df.set_index('datetime').resample('5Min').first()

    acc_mean = acc_resampled['vector_mag'].mean()
    acc_std = acc_resampled['vector_mag'].std()
    acc_min = acc_resampled['vector_mag'].min()
    acc_max = acc_resampled['vector_mag'].max()
    acc_q1g = acc_resampled['vector_mag'].quantile(0.25)
    acc_q3g = acc_resampled['vector_mag'].quantile(0.75)
    acc_skew = acc_resampled['vector_mag'].skew()


    ###### eda

    eda_df['datetime']=pd.to_datetime(eda_df['datetime'])
    eda_resampled = eda_df.set_index('datetime').resample('5Min').first()

    eda_mean = eda_resampled[' eda'].mean()
    eda_std = eda_resampled[' eda'].std()
    eda_min = eda_resampled[' eda'].min()
    eda_max = eda_resampled[' eda'].max()
    eda_q1g = eda_resampled[' eda'].quantile(0.25)
    eda_q3g = eda_resampled[' eda'].quantile(0.75)
    eda_skew = eda_resampled[' eda'].skew()

    # peak detection
    peaks_idx, properties = find_peaks(eda_resampled[' eda'], height=0, distance=4, prominence=0.3)
    peak_eda = len(peaks_idx)

    number_of_windows, peaks_in_window = find_peaks_in_rolling_window(eda_resampled, window_size='2H')
    peakEDA2hr_sum = peaks_in_window
    peakEDA2hr_mean = peaks_in_window / number_of_windows


    ###### hr

    if id == 1:
        hr_df['datetime']=pd.to_datetime(hr_df['datetime'], format='%m/%d/%y %H:%M')
    else:
        hr_df['datetime']=pd.to_datetime(hr_df['datetime'])
    hr_resampled = hr_df.set_index('datetime').resample('5Min').first()

    hr_mean = hr_resampled[' hr'].mean()
    hr_std = hr_resampled[' hr'].std()
    hr_min = hr_resampled[' hr'].min()
    hr_max = hr_resampled[' hr'].max()
    hr_q1g = hr_resampled[' hr'].quantile(0.25)
    hr_q3g = hr_resampled[' hr'].quantile(0.75)
    hr_skew = hr_resampled[' hr'].skew()


    ###### ibi

    ibi_df['datetime']=pd.to_datetime(ibi_df['datetime'])
    ibi_resampled = ibi_df.set_index('datetime').resample('5Min').first().interpolate(method='time')
    ibi_resampled[' ibi'] = ibi_resampled[' ibi'] * 1000 # 초 -> 밀리초 단위로

    maxHRV = ibi_resampled[' ibi'].max()
    minHRV = ibi_resampled[' ibi'].min()
    medianHRV = ibi_resampled[' ibi'].median()
    meanHRV = ibi_resampled[' ibi'].mean()
    sdnn = ibi_resampled[' ibi'].std()

    nn50 = 0.
    for i in range(len(ibi_resampled)-1):
        if np.abs(ibi_resampled.iloc[i+1][' ibi'] - ibi_resampled.iloc[i][' ibi']) > 50:
            nn50 +=1

    pnn50 = nn50/len(ibi_resampled)

    rmssd = 0.
    for i in range(len(ibi_resampled)-1):
        rmssd = np.abs(ibi_resampled.iloc[i+1][' ibi'] - ibi_resampled.iloc[i][' ibi'])**2
    rmssd = np.sqrt(rmssd) / len(ibi_resampled)


    ###### temp

    temp_df['datetime']=pd.to_datetime(temp_df['datetime'])
    temp_resampled = temp_df.set_index('datetime').resample('5Min').first()

    temp_mean = temp_resampled[' temp'].mean()
    temp_std = temp_resampled[' temp'].std()
    temp_min = temp_resampled[' temp'].min()
    temp_max = temp_resampled[' temp'].max()
    temp_q1g = temp_resampled[' temp'].quantile(0.25)
    temp_q3g = temp_resampled[' temp'].quantile(0.75)
    temp_skew = temp_resampled[' temp'].skew()


    ## results
    print('##################################')
    print(f'calories2hr : {calories2hr}')
    print(f'protein2hr : {protein2hr}')
    print(f'sugar2hr : {sugar2hr}')
    print(f'carbs2hr : {carbs2hr}')

    print(f'calories8hr : {calories8hr}')
    print(f'protein8hr : {protein8hr}')
    print(f'sugar8hr : {sugar8hr}')
    print(f'carbs8hr : {carbs8hr}')

    print(f'calories24hr : {calories24hr}')
    print(f'protein24hr : {protein24hr}')
    print(f'sugar24hr : {sugar24hr}')
    print(f'carbs24hr : {carbs24hr}')

    print(f'acc_mean : {acc_mean}')
    print(f'acc_std : {acc_std}')
    print(f'acc_min : {acc_min}')
    print(f'acc_max : {acc_max}')
    print(f'acc_q1g : {acc_q1g}')
    print(f'acc_q3g : {acc_q3g}')
    print(f'acc_skew : {acc_skew}')

    print(f'eda_mean : {eda_mean}')
    print(f'eda_std : {eda_std}')
    print(f'eda_min : {eda_min}')
    print(f'eda_max : {eda_max}')
    print(f'eda_q1g : {eda_q1g}')
    print(f'eda_q3g : {eda_q3g}')
    print(f'eda_skew : {eda_skew}')
    print(f'PeakEDA : {peak_eda}')
    print(f'PeakEDA2hr_sum : {peakEDA2hr_sum}')
    print(f'PeakEDA2hr_mean : {peakEDA2hr_mean}')

    print(f'hr_mean : {hr_mean}')
    print(f'hr_std : {hr_std}')
    print(f'hr_min : {hr_min}')
    print(f'hr_max : {hr_max}')
    print(f'hr_q1g : {hr_q1g}')
    print(f'hr_q3g : {hr_q3g}')
    print(f'hr_skew : {hr_skew}')

    print(f'maxHRV : {maxHRV}')
    print(f'minHRV : {minHRV}')
    print(f'medianHRV : {medianHRV}')
    print(f'meanHRV : {meanHRV}')
    print(f'SDNN : {sdnn}')
    print(f'NN50 : {nn50}')
    print(f'pNN50 : {pnn50}')
    print(f'RMSSD : {rmssd}')

    print(f'temp_mean : {temp_mean}')
    print(f'temp_std : {temp_std}')
    print(f'temp_min : {temp_min}')
    print(f'temp_max : {temp_max}')
    print(f'temp_q1g : {temp_q1g}')
    print(f'temp_q3g : {temp_q3g}')
    print(f'temp_skew : {temp_skew}')
    print('##################################')

    new_record = {
        'calories2hr': calories2hr,
        'protein2hr': protein2hr,
        'sugar2hr': sugar2hr,
        'carbs2hr': carbs2hr,
        'calories8hr': calories8hr,
        'protein8hr': protein8hr,
        'sugar8hr': sugar8hr,
        'carbs8hr': carbs8hr,
        'calories24hr': calories24hr,
        'protein24hr': protein24hr,
        'sugar24hr': sugar24hr,
        'carbs24hr': carbs24hr,
        'acc_mean': acc_mean,
        'acc_std': acc_std,
        'acc_min': acc_min,
        'acc_max': acc_max,
        'acc_q1g': acc_q1g,
        'acc_q3g': acc_q3g,
        'acc_skew': acc_skew,
        'eda_mean': eda_mean,
        'eda_std': eda_std,
        'eda_min': eda_min,
        'eda_max': eda_max,
        'eda_q1g': eda_q1g,
        'eda_q3g': eda_q3g,
        'eda_skew': eda_skew,
        'PeakEDA': peak_eda,
        'PeakEDA2hr_sum': peakEDA2hr_sum,
        'PeakEDA2hr_mean': peakEDA2hr_mean,
        'hr_mean': hr_mean,
        'hr_std': hr_std,
        'hr_min': hr_min,
        'hr_max': hr_max,
        'hr_q1g': hr_q1g,
        'hr_q3g': hr_q3g,
        'hr_skew': hr_skew,
        'maxHRV': maxHRV,
        'minHRV': minHRV,
        'medianHRV': medianHRV,
        'meanHRV': meanHRV,
        'SDNN': sdnn,
        'NN50': nn50,
        'pNN50': pnn50,
        'RMSSD': rmssd,
        'temp_mean': temp_mean,
        'temp_std': temp_std,
        'temp_min': temp_min,
        'temp_max': temp_max,
        'temp_q1g': temp_q1g,
        'temp_q3g': temp_q3g,
        'temp_skew': temp_skew
    }

    df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)
    

ID : 1 | target : 0


C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:49: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='2H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:55: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='8H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:61: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='24H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\1792054540.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in data.rolling(window=window_size):


##################################
calories2hr : 20893.600000000002
protein2hr : 1011.3000000000001
sugar2hr : 1240.8
carbs2hr : 2487.5999999999995
calories8hr : 38917.3
protein8hr : 1606.2
sugar8hr : 2642.400000000001
carbs8hr : 4957.200000000001
calories24hr : 92074.59999999998
protein24hr : 4206.999999999997
sugar24hr : 6144.199999999998
carbs24hr : 11382.299999999994
acc_mean : 64.01035563170277
acc_std : 5.315251848206103
acc_min : 28.035691537752374
acc_max : 111.61989070053778
acc_q1g : 62.76145313805282
acc_q3g : 64.77653896280658
acc_skew : 1.9510091083168808
eda_mean : 0.9408512084705883
eda_std : 1.9022207772357134
eda_min : 0.0
eda_max : 27.837559
eda_q1g : 0.135797
eda_q3g : 0.802353
eda_skew : 4.832521811562482
PeakEDA : 176
PeakEDA2hr_sum : 3872
PeakEDA2hr_mean : 1.4761723217689668
hr_mean : 73.44717043314502
hr_std : 16.438131272262012
hr_min : 47.57
hr_max : 176.35
hr_q1g : 60.73
hr_q3g : 81.69
hr_skew : 1.3140585318470634
maxHRV : 1625.0739999999998
minHRV : 312.514
m

C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:277: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)


ID : 2 | target : 0


...

##################################
calories2hr : 23178.4
protein2hr : 1682.8999999999999
sugar2hr : 670.1999999999998
carbs2hr : 1943.7999999999995
calories8hr : 41807.8
protein8hr : 2709.100000000001
sugar8hr : 1442.1999999999996
carbs8hr : 4328.200000000002
calories24hr : 87817.59999999998
protein24hr : 5652.600000000001
sugar24hr : 2766.1000000000004
carbs24hr : 8475.400000000001
acc_mean : 65.24360065991412
acc_std : 7.773974724947387
acc_min : 19.313207915827967
acc_max : 182.60339536821323
acc_q1g : 63.31863855381775
acc_q3g : 65.32036427495825
acc_skew : 5.787693070956214
eda_mean : 0.4494219392338177
eda_std : 0.9677084112879805
eda_min : 0.0
eda_max : 15.46084
eda_q1g : 0.14402075
eda_q3g : 0.268049
eda_skew : 6.707395766207298
PeakEDA : 83
PeakEDA2hr_sum : 1740
PeakEDA2hr_mean : 0.7432721059376335
hr_mean : 79.40847424042272
hr_std : 15.563898950635783
hr_min : 51.0
hr_max : 201.35
hr_q1g : 69.13
hr_q3g : 84.67250000000001
hr_skew : 2.0549794955730594
maxHRV : 1390.6889999999

C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:49: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='2H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:55: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='8H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:61: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='24H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\1792054540.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in data.rolling(window=window_size):


##################################
calories2hr : 28783.09999999999
protein2hr : 1230.3000000000002
sugar2hr : 1045.3999999999999
carbs2hr : 2779.3999999999987
calories8hr : 62054.2
protein8hr : 2699.6000000000004
sugar8hr : 2086.4000000000005
carbs8hr : 5600.699999999999
calories24hr : 152590.8000000001
protein24hr : 5961.700000000001
sugar24hr : 5356.100000000001
carbs24hr : 15090.200000000006
acc_mean : 64.30636546341354
acc_std : 8.531286549814734
acc_min : 20.83266665599966
acc_max : 180.71247881648904
acc_q1g : 62.369864518050704
acc_q3g : 64.91532946846992
acc_skew : 2.9985887034449203
eda_mean : 1.9149313798037613
eda_std : 3.4521805401287993
eda_min : 0.0
eda_max : 32.916756
eda_q1g : 0.36910125
eda_q3g : 1.7280595
eda_skew : 3.9382234253082036
PeakEDA : 179
PeakEDA2hr_sum : 3715
PeakEDA2hr_mean : 1.4255564082885648
hr_mean : 78.1399754601227
hr_std : 15.402973693681213
hr_min : 48.0
hr_max : 148.7
hr_q1g : 67.47
hr_q3g : 86.3
hr_skew : 0.9869163443590772
maxHRV : 1468.817
minH

C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:49: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='2H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:55: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='8H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\2445497863.py:61: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in fl_df.rolling(window='24H', on='time_begin'):
C:\Users\smyie\AppData\Local\Temp\ipykernel_7308\1792054540.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for window_data in data.rolling(window=window_size):


##################################
calories2hr : 41281.0
protein2hr : 1659.2000000000003
sugar2hr : 2158.3
carbs2hr : 5844.4
calories8hr : 71103.0
protein8hr : 3171.499999999999
sugar8hr : 3666.4999999999986
carbs8hr : 9864.499999999993
calories24hr : 174527.0
protein24hr : 7399.799999999998
sugar24hr : 9400.500000000004
carbs24hr : 24795.000000000022
acc_mean : 63.7463297841026
acc_std : 3.645762373668922
acc_min : 32.69556544854363
acc_max : 106.21675950620975
acc_q1g : 62.71363488110062
acc_q3g : 64.56779382943171
acc_skew : 2.0608790156669086
eda_mean : 0.7596999408955224
eda_std : 2.742281196818504
eda_min : 0.0
eda_max : 30.757719
eda_q1g : 0.046117
eda_q3g : 0.27491299999999996
eda_skew : 7.222315446598471
PeakEDA : 138
PeakEDA2hr_sum : 3022
PeakEDA2hr_mean : 1.050034746351633
hr_mean : 81.0596656716418
hr_std : 17.671136687029904
hr_min : 46.42
hr_max : 184.27
hr_q1g : 70.18
hr_q3g : 86.775
hr_skew : 1.6275212951850613
maxHRV : 1812.5830000000005
minHRV : 312.514
medianHRV : 73

In [ ]:
df.head()

In [ ]:
df.to_csv(r'./features.csv', sep='\t', encoding='utf-8', header='true')